In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random

In [4]:
ds = "ukr.txt"
english_txt = []
ukrainian_txt = []
english_ch = set()
ukrainian_ch = set()

with open(ds, "r", encoding = "utf-8") as f:
    lines = f.read().split("\n")

for line in lines[: min(20000, len(lines) - 1)]:
    english_from_file, ukrainian_from_file, attribution = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    ukrainian_from_file = "\t" + ukrainian_from_file + "\n"
    english_txt.append(english_from_file)
    ukrainian_txt.append(ukrainian_from_file)
    
    for ch in english_from_file:
        if ch not in english_ch:
            english_ch.add(ch)
            
    for ch in ukrainian_from_file:
        if ch not in ukrainian_ch:
            ukrainian_ch.add(ch)

In [10]:
for i in range(5):
    r_n = random.randint(0, len(english_txt))
    print(english_txt[r_n])
    print(ukrainian_txt[r_n])

I'm frugal.
	Я бережлива.

Don't pick it up.
	Не піднімай її.

It's night.
	Ніч.

I'm worried.
	Я турбуюся.

He slept an hour.
	Він проспав годину.



In [13]:
english_ch = sorted(list(english_ch))
ukrainian_ch = sorted(list(ukrainian_ch))

max_encoder_en_seq_length = max([len(en_txt) for en_txt in english_txt])
max_decoder_ukr_seq_length = max([len(ukr_txt) for ukr_txt in ukrainian_txt])

input_en_token_index = {ch: i for i, ch in enumerate(english_ch)}
target_ukr_token_index = {ch: i for i, ch in enumerate(ukrainian_ch)}

encoder_input_data = np.zeros((len(english_txt), max_encoder_en_seq_length, len(english_ch)), dtype="float32")
decoder_input_data = np.zeros((len(english_txt), max_decoder_ukr_seq_length, len(ukrainian_ch)), dtype="float32")
decoder_target_data = np.zeros((len(english_txt), max_decoder_ukr_seq_length, len(ukrainian_ch)), dtype="float32")
 
for i, (en, ukr) in enumerate(zip(english_txt, ukrainian_txt)):
    for t, ch in enumerate(en):
        encoder_input_data[i, t, input_en_token_index[ch]] = 1.0
    encoder_input_data[i, t + 1:, input_en_token_index[" "]] = 1.0
    
    for t, ch in enumerate(ukr):
        decoder_input_data[i, t, target_ukr_token_index[ch]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_ukr_token_index[ch]] = 1.0
            
    decoder_input_data[i, t + 1:, target_ukr_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_ukr_token_index[" "]] = 1.0

In [15]:
encoder_inputs = keras.Input(shape=(None, len(english_ch)))
encoder = keras.layers.LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]
decoder_inputs = keras.Input(shape=(None, len(ukrainian_ch)))
decoder_lstm = keras.layers.LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = keras.layers.Dense(len(ukrainian_ch), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [16]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

my_callbacks = [keras.callbacks.EarlyStopping(patience = 50)]

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = 64, epochs = 100, validation_split=0.2,)

model.save("s2s")


Epoch 1/100


250/250 [==============================] - 26s 92ms/step - loss: 1.1241 - accuracy: 0.7419 - val_loss: 0.9815 - val_accuracy: 0.7449
Epoch 2/100
250/250 [==============================] - 23s 91ms/step - loss: 0.7327 - accuracy: 0.8007 - val_loss: 0.7722 - val_accuracy: 0.7843
Epoch 3/100
250/250 [==============================] - 22s 89ms/step - loss: 0.6462 - accuracy: 0.8168 - val_loss: 0.7229 - val_accuracy: 0.7929
Epoch 4/100
250/250 [==============================] - 22s 90ms/step - loss: 0.5922 - accuracy: 0.8287 - val_loss: 0.6848 - val_accuracy: 0.8029
Epoch 5/100
250/250 [==============================] - 23s 91ms/step - loss: 0.5618 - accuracy: 0.8363 - val_loss: 0.6565 - val_accuracy: 0.8101
Epoch 6/100
250/250 [==============================] - 22s 90ms/step - loss: 0.5367 - accuracy: 0.8431 - val_loss: 0.6319 - val_accuracy: 0.8178
Epoch 7/100
250/250 [==============================] - 23s 90ms/step - loss: 0.5159 - accuracy: 0.8491 - val_loss: 0.6134 - val

INFO:tensorflow:Assets written to: s2s\assets


In [18]:
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output 
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(256,))
decoder_state_input_c = keras.Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_en_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_ukr_token_index.items())

In [25]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, len(ukrainian_ch)))
    target_seq[0, 0, target_ukr_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_ukr_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, len(ukrainian_ch)))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [27]:
for seq_index in range(50):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
    print("Input sentence:", english_txt[seq_index])
    print("Decoded sentence:", decoded_sentence)
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")

1/1 [==============================] - 0s 19ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: Go.
Decoded sentence: Йди.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1/1 [==============================] - 0s 20ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: Hi.
Decoded sentence: Привіт.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1/1 [==============================] - 0s 21ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: Hi.
Decoded sentence: Привіт.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1/1 [==============================] - 0s 21ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: Hi.
Decoded sentence: Привіт.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1/1 [==============================] - 0s 19ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: Run!
Decoded sentence: Біжіть!

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1/1 [==============================] - 0s 20ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: